In [3]:
# step 1: import dem libraries kiddos!
import json
import pandas as pd
import numpy as np
import nltk as nltk
import spacy
import os
import re 
import sklearn 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from collections import Counter
import nltk
from readability import Readability  #https://github.com/cdimascio/py-readability-metrics
stop_words = nltk.corpus.stopwords.words('english')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
nltk.download('stopwords')
from nltk.corpus import stopwords
stpwds = set(stopwords.words('english'))
#nltk.download('wordnet')
nlp = spacy.load('C://Users//seanp//Anaconda3//Lib//site-packages//en_core_web_sm//en_core_web_sm-2.0.0')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import *
porter_stemmer = PorterStemmer()
import nltk.data
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tokenize import TreebankWordTokenizer
treebank_tokenizer = TreebankWordTokenizer()
from nltk.util import bigrams 



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\seanp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# import the diaglogue Json Files!!!!!

wd = os.getcwd() 
# grab the locations of those lil jsons
json_files = [pos_json for pos_json in os.listdir(wd + '\\diag_jsons\\') if pos_json.endswith('.json')]

# read those guys in!

# the first step will be to READ each file, then the NEXT step will be to store it in something that will EVENTUALLY 
# be converted into a dataframe (actually, series)

# so I reckon the way to start this bad boy off is to just store them all in a SHINY NEW DICTIONARY

# and let the key be the movie name  


In [63]:
# I commented out an example. 
with open(wd+ '\\diag_jsons\\'+json_files[0], "r") as read_file:
    data = json.load(read_file)
json_files[0].replace('.json','')

KAT____10-Things-I-Hate-About-You_script           leave_it it_i i_said said_, ,_leave leave_it i...
PATRICK____10-Things-I-Hate-About-You_script       i_missed missed_you you_. it_was was_a a_bratw...
BIANCA____10-Things-I-Hate-About-You_script        did_you you_change change_your your_hair hair_...
CAMERON____10-Things-I-Hate-About-You_script       i_do do_n't n't_think think_so so_, ,_ma'am ma...
MICHAEL____10-Things-I-Hate-About-You_script       you_the the_new new_guy guy_? c'mon_. i_'m 'm_...
JOEY____10-Things-I-Hate-About-You_script          as_opposed opposed_to to_a a_bitter bitter_sel...
WALTER____10-Things-I-Hate-About-You_script        i_hope hope_dinner dinner_'s 's_ready ready_be...
MANDELLA____10-Things-I-Hate-About-You_script      uh_, ,_yeah yeah_, ,_i i_read read_it it_all a...
MISS PERKY____10-Things-I-Hate-About-You_script    i_'m 'm_sure sure_you you_wo wo_n't n't_find f...
CHASTITY____10-Things-I-Hate-About-You_script      no_. who_? great_is is_he he_oily oily_o

In [5]:
# found this on stack overflow
def remove_multiple_strings(cur_string, replace_list,low= False):
    for cur_word in replace_list:
        if low == False:
            cur_string = cur_string.replace(cur_word, '')
        else:
            cur_string = cur_string.replace(cur_word.lower(), '')
    return cur_string

In [7]:
# Original Process with LEMMATIZATION but no bigrams (we do remove all character names, we will also want to get rid of stopwords )
mov_dict = {}
char_series = pd.Series()
for fil in json_files:
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
            lemz += " ".join(lemmatized_words) + ' '
            lemz = [w for w in treebank_tokenizer.tokenize(lemz) if w not  in stpwds]
            lemz = [tok for tok in nlp(lemz) if tok.pos_ in ['NOUN','VERB','ADV','ADJ'] ]
            
        char_series.at[char+'____'+fil.replace('.json','')] = lemz
        
        # we get rid of those pesky \n[number] patterns that denote a line change 
        # which I suppose I can add back IN as a second column if necessary , but that's just going to be unnecessary for the clustering!

# I'm guessing that every thime there is a new line it will go with that kind of scheme 
# so I suppose my job will be to remove those bois if they still occur after tokenization.... hoping they just dissapear, lmao !!!

# also, very nice, I can iterate over the ['dialogues.keys()']
# this is VERY fun!!!! 


In [81]:
feat_frame
#thingz = sentence_tokenizer.tokenize(miniboi[0])
# so... we need to do this for every sentence.... YAY.... 
#lemz = ''
# I think I need to get rid of punctuation unless lemmatization gets rid of it 
#for sentence in thingz:
#    toks = treebank_tokenizer.tokenize(sentence)
#    lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
#    lemz += " ".join(lemmatized_words) + ' '


In [30]:
#lemz

In [31]:
# for examples sake... run from the line that says mniboi  until the one that says miniboi[0]
#miniboi

#miniboi[0] = re.sub("\\n\[\d+\]",'',miniboi[0])
#         char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])

#miniboi[0]

# you want to lemmatize the the text or tokenize etc, 

# tokenize by sentence 1st...then apply the lemmatization to each sentence

# then we have to find a way to concatenate or .join everything together across all of the lists!!! 
#miniboi[0]
# Do some Data preprocessing on the text
# maybe tokenizing, stemming or lemmatizing will get rid of the bad woyds!
# so now I know to do this to the text to preprocess it, again, I can do it in the initial loop, or after. 
# it's kinda whatever. 

# time to lemmatize!!!!
# or friggin tokenize, at least!
#thingz = sentence_tokenizer.tokenize(miniboi[0])
#print(thingz)
#lemz = []
#for sentence in thingz:
#    lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in sentence]
#    lemz += lematized_words
#lemmatized_words
#" ".join(["John", "Charles", "Smith"])

# Time to get those term counts!!!!!
#lemz
#miniboi[0]

In [14]:
# LSA, LSA!!!!
# wondeirng what the count_vecotrizer does??? Here's the basic documentation: 
# Convert a collection of text documents to a matrix of token counts
#This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.
#If you do not provide an a-priori dictionary and you do not use an analyzer that does some kind of feature selection then the number of features will be equal to the vocabulary size found by analyzing the data.
count_vectorizer = CountVectorizer(stop_words='english', max_features=10000)
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
#display(document_term_matrix)

C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [33]:
# Let's reduce the dimensionality of that Document term matrix and do some clustering! 

# I"m doing this for 2 reasons: 1, I essentially copy pasted from Stephen 

#2:.... we have a lot of data.... let's make the run time as fast as possible!


In [32]:
# Define helper functions, These are from Stephen
def get_keys(topic_matrix):
    '''returns an integer list of predicted topic categories for a given topic matrix'''
    keys = []
    for i in range(topic_matrix.shape[0]):
        keys.append(topic_matrix[i].argmax())
    return keys
###################################################################################
def keys_to_counts(keys):
    '''returns a tuple of topic categories and their accompanying magnitudes for a given list of keys'''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

##############################################################################
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''returns a list of n_topic strings, where each string contains the n most common 
        words in a predicted category, in order'''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [36]:
count_vectorizer = CountVectorizer(ngram_range = (4,4))
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)
for i in range(5,31):
    n_topics = i

    lsa_model = TruncatedSVD(n_components=n_topics)
    lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
   # display(lsa_topic_matrix)

    lsa_keys = get_keys(lsa_topic_matrix)
    lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

    top_n_words_lsa = get_top_n_words(5, lsa_keys, document_term_matrix, count_vectorizer)

    for i in range(len(top_n_words_lsa)):
        print("Topic {}: ".format(i), top_n_words_lsa[i])
    print('********')

C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


<11075x4856542 sparse matrix of type '<class 'numpy.int64'>'
	with 5480525 stored elements in Compressed Sparse Row format>

Topic 0:  deblois sanders final draft draft 02 13 2010 sanders final draft 02 dragon deblois sanders final final draft 02 13
Topic 1:  what are you doing you re going to you re gon na we re going to what do you mean
Topic 2:  pink revision 20 07 pink revision 19 07 yellow revision 20 07 real slow real slow one of the most
Topic 3:  am am am am approaching newark station newark to camera to camera now approaching newark station station newark station next
Topic 4:  la la la la the scene dissolve to the scene cut to scene cut to the in front of him
********
Topic 0:  sanders final draft 02 deblois sanders final draft draft 02 13 2010 dragon deblois sanders final final draft 02 13
Topic 1:  what are you doing you re going to you re gon na what do you mean we re going to
Topic 2:  red rum red rum rum red rum red you may be sure in the midst of in the world the
Topic 3:  to camera to camera camera to camera to in and out of in the history of the heart of the
Topic 4:  pink revision 20 07 fr

Topic 7:  blue valentine official green green script of 09 valentine official green script official green script of no subtitle japanese no
Topic 8:  yes mr yes mr seen evil dead ii nothing to do with mr yes mr yes what do you mean
Topic 9:  thank you thank you thank you so much do you hear the my name is cohen you do have to
Topic 10:  we ll have to you ll have to the edge of the he shake his head she shake her head
Topic 11:  you re looking for we re going to what we re doing to be able to oh dear oh dear
Topic 12:  4th blue revision 03 blue revision 03 26 revision 03 26 10 and the warrior three the side of the
Topic 13:  am am am am camera to camera to you re gon na to camera to camera what do you mean
Topic 14:  from original script sequence sequence from original script script sequence from original original script sequence from ted from original script
Topic 15:  cedar rapid 12 09 red rum red rum rum red rum red will overload in minute engine will overload in
Topic 16:  you re go

Topic 15:  we ve got to pick up the phone we just have to there only one way ve got to think
Topic 16:  what do you mean it thrill to be keep an eye on her against the wall to keep an eye
Topic 17:  pink revision 20 07 pink revision 19 07 head of mall security to camera to camera yellow revision 20 07
Topic 18:  you know what mean none of my business it none of my get the fuck out the fuck out of
Topic 19:  we re gon na you re gon na what do you think you want me to what do you mean
Topic 20:  cedar rapid 12 09 from original script sequence sequence from original script original script sequence from script sequence from original
Topic 21:  red rum red rum rum red rum red and then you re then you re gone you re going to
Topic 22:  revision 03 26 10 blue revision 03 26 4th blue revision 03 and the warrior three what are you doing
Topic 23:  what do you mean what the matter with you re going to we re going to the side of the
Topic 24:  no no no no that ai no crime no crime that ai crime t

In [36]:
# Now we will do LSA with tokenization instead of lemmatization, just so that the outputted words make more sense!
# Hopefully there's a way to do this with sequences of 2-3 words, because some of these are gibberish.


NameError: name 'char_series' is not defined

In [6]:
# original but with just tokenization, no bigrams
mov_dict = {}
char_series = pd.Series()
for fil in json_files:
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            #lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
            lemz += " ".join(toks) + ' '
        char_series.at[char+'____'+fil.replace('.json','')] = lemz
        # we get rid of those pesky \n[number] patterns that denote a line change 
        # which I suppose I can add back IN as a second column if necessary , but that's just going to be unnecessary for the clustering!

# I'm guessing that every thime there is a new line it will go with that kind of scheme 
# so I suppose my job will be to remove those bois if they still occur after tokenization.... hoping they just dissapear, lmao !!!

# also, very nice, I can iterate over the ['dialogues.keys()']
# this is VERY fun!!!! 


In [ ]:
# this is where I only keep verbs adverbs and adjectives
# I will also get rid of any remaining stopwords. 
mov_dict = {}
char_series = pd.Series()
for fil in json_files:
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            #lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
            lemz += " ".join(toks) + ' '
            lemz = [w for w in treebank_tokenizer.tokenize(lemz) if w not  in stpwds]
            #print(lemz)
            lemz =  " ".join(lemz)
            lemz = [str(tok) for tok in nlp(lemz) if tok.pos_ in ['NOUN','VERB','ADV','ADJ'] ]

            lemz =  " ".join(lemz) + ' '
        char_series.at[char+'____'+fil.replace('.json','')] = lemz

In [26]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=10000)
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)

n_topics = i

lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
   # display(lsa_topic_matrix)

lsa_keys = get_keys(lsa_topic_matrix)
lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

top_n_words_lsa = get_top_n_words(20, lsa_keys, document_term_matrix, count_vectorizer)

for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i), top_n_words_lsa[i])


C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


<11075x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 2789943 stored elements in Compressed Sparse Row format>

Topic 0:  youhhh ihhh hhh ithhh hhhi dohhhn thathhh mehhh hhhyou ll hhhand ihhhdo whathhh wehhh ve nohhh inhhhthe herehhh hehhh therehhh
Topic 1:  hhh hhhthe hhhand hhhi uhhh ofhhhthe inhhhthe youhhh ithhh hhhyou hhhwe tohhhthe herehhh onhhhthe hhha mr hhhu hhhhe ihhh thathhh
Topic 2:  ofhhhthe inhhhthe tohhhthe onhhhthe hhhand hhhthe athhhthe ithhh himhhh intohhhthe fromhhhthe thehhhdoor hhh andhhhthe hhha outhhhof hhhhe uphhh hehhh doorhhh
Topic 3:  hhhand hhhi ofhhhthe hhhthe inhhhthe hhhbut mehhh ihhhwas tohhhthe ithhhwas ihhham hhh andhhhthe ihhhhad hhhhe tohhhbe ithhh hhhas thathhhi forhhhthe
Topic 4:  yeahhhh ithhh thathhh heyhhh righthhh whathhh uphhh lethhh hehhh dohhhn ohhhh gohhh wehhh onhhh shhhgo hhhwe shhhthe gonhhhna hhhyeah hhhwhat
Topic 5:  youhhh hhhyou rumhhh youhhhare hhhand youhhhknow thathhh wellhhh manhhh ofhhhthe tohhhyou timeshhh athhhall thishhh ithhhis dohhhn youhhhhave whathhh youhhhready thankhhhyou
Topic 6:  aihhhn thhhno thathhhai crimehhh ihhh gonhhhna m

In [ ]:
# THERE IS A graph algo called infomap... code available...all you provide it is a graph????? 

# i.e. character1 movie1 compared to character 7 movie 3... put the similarity in there.... 

# benefit of info map is it will just see it as 1 big grph and builds communities inside of graph... 


# pici villain from jaems bond movie... pick departed.. if you find bond character 1 and 

# we can remove stopwords... before 

# we can go oline and downlaoad it.... 

# mapequation.org.. it's a C++ implementation..... so we put in the graph..... 

In [51]:
# create a new dictionary where each key will be the movie and each value will contain all of the 
mov_dict = {}
char_series = pd.Series()
for fil in json_files:
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            bigrams = nltk.bigrams(toks)
            bgs = ["_".join(gram) for gram in bigrams ]
                
                    
            #lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
    
            lemz += " ".join(bgs) + ' '
        char_series.at[char+'____'+fil.replace('.json','')] = lemz
        # we get rid of those pesky \n[number] patterns that denote a line change 
        # which I suppose I can add back IN as a second column if necessary , but that's just going to be unnecessary for the clustering!

# I'm guessing that every thime there is a new line it will go with that kind of scheme 
# so I suppose my job will be to remove those bois if they still occur after tokenization.... hoping they just dissapear, lmao !!!

# also, very nice, I can iterate over the ['dialogues.keys()']
# this is VERY fun!!!! 


In [20]:
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)

<11075x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 1406463 stored elements in Compressed Sparse Row format>

In [33]:
count_vectorizer = CountVectorizer()
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)
for i in range(5,31):
    n_topics = i

    lsa_model = TruncatedSVD(n_components=n_topics)
    lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
   # display(lsa_topic_matrix)

    lsa_keys = get_keys(lsa_topic_matrix)
    lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

    top_n_words_lsa = get_top_n_words(20, lsa_keys, document_term_matrix, count_vectorizer)

    for i in range(len(top_n_words_lsa)):
        print("Topic {}: ".format(i), top_n_words_lsa[i])
    print('********')

C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


<11075x844605 sparse matrix of type '<class 'numpy.int64'>'
	with 5112881 stored elements in Compressed Sparse Row format>

Topic 0:  you_ i_ it_ _i re do_n that_ me_ _you ll _and i_do what_ we_ ve no_ in_the here_ he_ there_
Topic 1:  of_the in_the _the to_the _and on_the at_the him_ it_ into_the from_the _a _he and_the out_of the_door up_ he_ her_ them_
Topic 2:  _and _i of_the _the in_the _6 revision_ 07 _but 20 me_ to_the i_am i_was and_the pink_revision it_ it_was yes_ _a
Topic 3:  yeah_ rum_ red_rum _you hey_ ai_n right_ it_ that_ai let_ t_no no_crime that_ crime_ what_ you_know mack_ again_ on_ here_
Topic 4:  you_ _you re mr you_are times_ all_times gentlemen_ at_all sir_ ready_ this_ order_ here_ you_ready thank_you mr_vaughn re_a _let please_
********
Topic 0:  you_ i_ it_ _i re do_n that_ me_ _you ll _and i_do what_ we_ ve no_ in_the here_ he_ there_
Topic 1:  of_the in_the _the to_the _and on_the at_the him_ it_ into_the _a from_the _he and_the out_of the_door up_ he_ her_ them_
Topic 2:  _and _i of_the _the in_the _6 07 revision_ _but 20 me_ to_the i_am i_was pink_revision and_the it_ it_was _a

KeyboardInterrupt: 

In [70]:
# playing with spacy to get the passive voice tokens 

nlp = spacy.load('C://Users//seanp//Anaconda3//Lib//site-packages//en_core_web_sm//en_core_web_sm-2.0.0')
each_sentence =  feat_frame.iloc[0,0]
doc=nlp(each_sentence)

passive_toks=[tok for tok in doc if (tok.dep_ == "nsubjpass" or tok.dep_ =="auxpass") ]
if passive_toks != []:
    print(passive_toks)
    
for tok in doc:
    if tok.pos_ in ['NOUN','VERB','ADV','ADJ']:
        print(tok.pos_)


[i, you, 're, i, getting, you, 're, i, be, you, were, i, got, being, you, were, am, i]
VERB
VERB
VERB
ADV
VERB
ADV
ADV
VERB
ADJ
NOUN
NOUN
VERB
ADV
NOUN
ADJ
NOUN
NOUN
ADV
ADV
ADJ
NOUN
ADV
VERB
VERB
ADV
ADV
ADV
ADV
VERB
ADJ
ADJ
NOUN
NOUN
VERB
NOUN
NOUN
VERB
NOUN
NOUN
VERB
NOUN
NOUN
NOUN
VERB
ADJ
ADJ
NOUN
NOUN
NOUN
NOUN
NOUN
ADJ
VERB
ADJ
NOUN
VERB
ADV
ADJ
NOUN
ADV
VERB
VERB
NOUN
ADV
NOUN
ADJ
ADV
VERB
ADJ
ADJ
NOUN
NOUN
VERB
VERB
NOUN
VERB
NOUN
NOUN
ADV
VERB
ADJ
NOUN
VERB
ADJ
VERB
VERB
ADJ
NOUN
VERB
ADJ
VERB
ADV
ADJ
VERB
NOUN
ADV
ADJ
NOUN
NOUN
VERB
VERB
VERB
ADV
ADJ
NOUN
VERB
NOUN
VERB
VERB
NOUN
ADJ
NOUN
VERB
ADV
ADJ
VERB
VERB
VERB
ADJ
NOUN
VERB
NOUN
ADJ
ADJ
NOUN
VERB
VERB
ADV
ADJ
ADJ
NOUN
NOUN
NOUN
VERB
NOUN
NOUN
VERB
ADV
ADJ
NOUN
VERB
VERB
VERB
VERB
ADV
VERB
VERB
ADV
VERB
ADV
VERB
VERB
VERB
VERB
VERB
NOUN
ADJ
NOUN
VERB
NOUN
VERB
ADJ
NOUN
VERB
ADJ
ADJ
NOUN
VERB
ADJ
NOUN
ADJ
NOUN
NOUN
NOUN
VERB
ADV
ADV
ADJ
VERB
NOUN
NOUN
ADV
VERB
VERB
NOUN
NOUN
VERB
ADJ
NOUN
NOUN
VERB
ADJ
ADJ
ADV
VERB
ADV
A

In [45]:
cosDF = pd.Series(index = char_series.index)
cosDF = pd.DataFrame(cosDF)
for movChar in char_series.index:
    cosDF[str(movChar)] = pd.Series(np.nan,index = char_series.index)
cosDF.drop(cosDF.columns[0], axis=1,inplace = True)  

In [ ]:
#### Testing out bigrams code char_series.head()

mytest = char_series[0]
sentTest = sentence_tokenizer.tokenize(mytest)
sentences_words = [treebank_tokenizer.tokenize(sentence) for sentence in sentTest]
all_tokens = [word for sentence in sentences_words for word in sentence]
bigrams = nltk.bigrams(all_tokens)
new_text = ''
bgs = ["***".join(gram) for gram in bigrams ]
bigramz = " ".join(bgs)
bigramz


# nouns verbs adverbs adjectives... 

In [49]:
# Playing with vader sentiment 

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
#miniboi = pd.Series()
#for i in mov_dict['10-Things-I-Hate-About-You_script']['dialogues'].keys():
#    miniboi.at[i+'____''10-Things-I-Hate-About-You_script'] = mov_dict['10-Things-I-Hate-About-You_script']['dialogues'][i]
#miniboi
feat_frame = pd.DataFrame(char_series)
feat_frame['overall_polarity'] = ['WRONG']* char_series.shape[0]
feat_frame.columns = ['dialogue','overall_polarity']
feat_frame.head()
for index, row in feat_frame.iterrows():
    
       feat_frame.loc[index,row.index[1]] = analyser.polarity_scores(row[0])['compound']


# ok, so I suppose we will need to figure out how we will create the new index for the Pandas.Series
# heck, I suppose I could even add a movie title AFTER... but meh, I'm lazy... I think I'll just leave it as the series for now! 


In [86]:
x_nostop = [w for w in treebank_tokenizer.tokenize(tizz) if w not  in stpwds] 

NameError: name 'char_series' is not defined

In [8]:
#### start here for the clustering features

char_series.head()
# alright, so analyzing the sentiment of what they are saying ABOUT each other is going to take a loooong fucking time 
# so how about doing 1: basic sentiment 
# FUCK I can't do passive voice bc of spacy... SHIT 
# maybe I ask pedro to do it in the cloud
# so .... count # of stopwords... 
# count number of sentences???? 
# need better.... 
# check for b-b- for stutters??? 
# that's toufh 

KAT____10-Things-I-Hate-About-You_script        leave it i said , leave it ! why did n't we ju...
PATRICK____10-Things-I-Hate-About-You_script    i missed you . it was a bratwurst . i was eati...
BIANCA____10-Things-I-Hate-About-You_script     did you change your hair ? you might wan na th...
CAMERON____10-Things-I-Hate-About-You_script    i do n't think so , ma'am so they tell me ... ...
MICHAEL____10-Things-I-Hate-About-You_script    you the new guy ? c'mon . i 'm supposed to giv...
dtype: object

In [69]:
#for index,row in pd.DataFrame(char_series).iterrows():
    
# number of first person pronouns

# % of sentences with hedges
# sentiment of the verbs 
# number of verbs per sentence? 
# look for concession dictionary
# look for FILLER/PAUSE dictionaries... (those might be stopwords) 
# look for contrast dictionaries..... 
# look for OPINION... 

hedge_words = ['largely','generally',
'often','rarely','sometimes','frequently','occasionally','seldom','usually','most','several','some','almost','practically',
'apparently','virtually','basically','approximately','roughly','somewhat','somehow','partially','actually','like','something',
'someone','somebody','somewhere',
'assume','assumes','assumed','appear','appears','appeared','seem','seems',
'seemed','suppose','supposes','supposed','guess','guesses','guessed','estimate','estimates','estimated',
'speculate',
'speculates',
'speculated',
'suggest',
'suggests',
'suggested',
'may',
'could',
'should',
'might',
'surely',
'probably',
'likely',
'maybe',
'perhaps',
'unsure',
'probable',
'unlikely',
'possibly',
'possible',
'read',
'say',
'says',
'looks like',
'look like',
"do n't know",
"necessarily",
"kind of",
"much",
"bunch",
"couple",
"few",
"little",
"really",
"and all that",
"and so forth",
"et cetera",
"if i 'm understanding you correctly",
"something or other",
"so far",
"at least",
"around",
"can",
"effectively",
"evidently",
"fairly",
"hopefully",
"in general",
"mainly",
"more or less",
"mostly",
"overall",
"presumably",
"pretty",
"quite clearly",
"quite",
"rather",
"sort of",
"supposedly",
"tend",
"appear to be",
"doubt",
"be sure",
"indicate",
"must",
"would",
"certainly",
"definitely",
"clearly",
"conceivably",
"certain",
"definite",
"clear",
"assumption"]




feels = ['think','thinks','thought','believe','believed','believes','consider','considers','considered',"in my mind",
"in my opinion",
"their impression",
"my impression",
"my understanding",
"my thinking is",
"my understanding is",
"in my view", 'my stance', 'my two cents',
'if you ask me']
p = re.compile('(u+h+)|(u+m+)|(e+r+m*)|(\.\.\.)|(h+m+)|(huh)')
p2 = re.compile('(f+u+c+k)|(s+h+i+t)|(d+a+m+n)|(b+i+t+c+h)|(b+a+s+t+a+r+d+)|(c+o+c+k)|(d+i+c+k)|(n+i+g+a+)|(b+a+l+l+s)')


In [63]:
import re
char_series = pd.DataFrame(char_series)

In [71]:
from readability import Readability  #https://github.com/cdimascio/py-readability-metrics


In [82]:

# so... I guess I could just tokenize every sentnece and go from there??? 
# nononnonon for regex, we HAVE to actually do the word tokenizer... 
countar = 0 
for index, row in char_series.iterrows():
    countar +=1
    if countar == 10:
        print('we processed 10')
    if countar == 100:
        print('we processed 100')
    if countar == 500:
        print('we processed 500')
    words = word_tokenize(row[0])

    sentz = sentence_tokenizer.tokenize(row[0])
        # iterate though each string inside of the list of sentences       
        #num_passive = 0
        #coref_sents = 0 
    num_fill = 0
    num_feels = 0
    num_hw = 0
    for sentence in sentz:
        feelList = [feelWyd for feelWyd in feels if feelWyd in sentence]
        if feelList == []:
            num_feels +=0
        else:
            num_feels += len(feelList)
        hwList = [hwWyd for hwWyd in hedge_words if hwWyd in sentence]
        if hwList == []:
            num_hw += 0
        else:
            num_hw += len(hwList)
            
        fillWyds = [word for word in word_tokenize(sentence) if p.fullmatch(word) != None]
        if fillWyds == []:
            num_fill +=0
        else:
            num_fill+= len(fillWyds)
    char_series.loc[index,'tot_feel'] = num_feels
    char_series.loc[index,'tot_fill'] = num_fill
    char_series.loc[index,'tot_hw'] = num_hw

    punct_sings = ".!?,;:-_--\'\'\'``..."
    words_stemmed = [porter_stemmer.stem(word) for word in words if word not in punct_sings]
        # Gets the number of unique stems
    char_series.loc[index,'uniqwords'] = len(set(words_stemmed))
        
        # FEAT #2: Flesch Kincaid Read Level per character --> Categorical
        # https://github.com/cdimascio/py-readability-metrics
  
        
        # FEAT #3: Number of Stop Words per character --> Numerical
    char_series.loc[index,'stopwords']  =  len(set([w for w in words if w.lower() in stpwds]))
    char_series.loc[index,'polarity'] = analyser.polarity_scores(row[0])['compound']

      
        
        
        
    

we processed 10
we processed 100
we processed 500


In [79]:
char_series['tot_feel'] = np.zeros(char_series.shape[0])
char_series['tot_fill'] = np.zeros(char_series.shape[0])
char_series['tot_hw'] = np.zeros(char_series.shape[0])
char_series['stopwords'] = np.zeros(char_series.shape[0])
char_series['uniqwords'] = np.zeros(char_series.shape[0])
char_series['polarity'] = np.zeros(char_series.shape[0])
char_series['cursewords'] = np.zeros(char_series.shape[0])
char_series['FK'] = np.zeros(char_series.shape[0])
char_series.loc[index,'polarity'] = np.zeros(char_series.shape[0])


ValueError: Must have equal len keys and value when setting with an iterable

In [68]:
char_series.tot_hw

KAT____10-Things-I-Hate-About-You_script           0.0
PATRICK____10-Things-I-Hate-About-You_script       0.0
BIANCA____10-Things-I-Hate-About-You_script        0.0
CAMERON____10-Things-I-Hate-About-You_script       0.0
MICHAEL____10-Things-I-Hate-About-You_script       0.0
JOEY____10-Things-I-Hate-About-You_script          0.0
WALTER____10-Things-I-Hate-About-You_script        0.0
MANDELLA____10-Things-I-Hate-About-You_script      0.0
MISS PERKY____10-Things-I-Hate-About-You_script    0.0
CHASTITY____10-Things-I-Hate-About-You_script      0.0
SHARON____10-Things-I-Hate-About-You_script        0.0
BRUCE____10-Things-I-Hate-About-You_script         0.0
TEACHER____10-Things-I-Hate-About-You_script       0.0
COLE____12-Monkeys_script                          0.0
RAILLY____12-Monkeys_script                        0.0
JEFFREY____12-Monkeys_script                       0.0
ASTROPHYSICIST____12-Monkeys_script                0.0
RASPY VOICE____12-Monkeys_script                   0.0
FALE____12

In [81]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [83]:
char_series.head()

,0,tot_feel,tot_fill,tot_hw,stopwords,polarity,cursewords,FK,uniqwords
KAT____10-Things-I-Hate-About-You_script,"leave it i said , leave it ! why did n't we ju...",18.0,0.0,87.0,97.0,-0.9984,0.0,0.0,603.0
PATRICK____10-Things-I-Hate-About-You_script,i missed you . it was a bratwurst . i was eati...,6.0,13.0,60.0,86.0,-0.9952,0.0,0.0,465.0
BIANCA____10-Things-I-Hate-About-You_script,did you change your hair ? you might wan na th...,13.0,9.0,75.0,85.0,0.9968,0.0,0.0,418.0
CAMERON____10-Things-I-Hate-About-You_script,"i do n't think so , ma'am so they tell me ... ...",8.0,8.0,58.0,71.0,0.9951,0.0,0.0,316.0
MICHAEL____10-Things-I-Hate-About-You_script,you the new guy ? c'mon . i 'm supposed to giv...,7.0,3.0,46.0,83.0,0.9984,0.0,0.0,419.0


In [93]:
char_series.head(15)

,0,tot_feel,tot_fill,tot_hw,stopwords,polarity,cursewords,FK,uniqwords
KAT____10-Things-I-Hate-About-You_script,"leave it i said , leave it ! why did n't we ju...",18.0,0.0,87.0,97.0,-0.9984,0.0,0.0,603.0
PATRICK____10-Things-I-Hate-About-You_script,i missed you . it was a bratwurst . i was eati...,6.0,13.0,60.0,86.0,-0.9952,0.0,0.0,465.0
BIANCA____10-Things-I-Hate-About-You_script,did you change your hair ? you might wan na th...,13.0,9.0,75.0,85.0,0.9968,0.0,0.0,418.0
CAMERON____10-Things-I-Hate-About-You_script,"i do n't think so , ma'am so they tell me ... ...",8.0,8.0,58.0,71.0,0.9951,0.0,0.0,316.0
MICHAEL____10-Things-I-Hate-About-You_script,you the new guy ? c'mon . i 'm supposed to giv...,7.0,3.0,46.0,83.0,0.9984,0.0,0.0,419.0
JOEY____10-Things-I-Hate-About-You_script,as opposed to a bitter self-righteous hag who ...,3.0,0.0,17.0,71.0,0.9830,0.0,0.0,249.0
WALTER____10-Things-I-Hate-About-You_script,i hope dinner 's ready because i only have ten...,3.0,0.0,14.0,72.0,0.9483,0.0,0.0,232.0
MANDELLA____10-Things-I-Hate-About-You_script,"uh , yeah , i read it all have n't you ? your ...",4.0,2.0,20.0,56.0,0.9972,0.0,0.0,198.0
MISS PERKY____10-Things-I-Hate-About-You_script,i 'm sure you wo n't find padua any different ...,1.0,2.0,15.0,52.0,0.8633,0.0,0.0,176.0
CHASTITY____10-Things-I-Hate-About-You_script,"no . who ? great is he oily or dry ? , i do n'...",2.0,0.0,3.0,32.0,0.7131,0.0,0.0,69.0


In [85]:
char_series.to_csv("character_features.csv")

In [146]:
diag_counts = {}
for i, char in enumerate(char_series.index[:-1]):
    movie_title = char[char.find("_")+4:]
    next_movie = char_series.index[i+1][char_series.index[i+1].find("_")+4:]
    if next_movie != movie_title:
        diag_counts[movie_title][char] = len(char_series.loc[char][0])
        diag_counts[next_movie] = {}
    else:
        if movie_title not in diag_counts:
            diag_counts[movie_title] = {}
            diag_counts[movie_title][char] = len(char_series.loc[char][0])
        else:
            diag_counts[movie_title][char] = len(char_series.loc[char][0])

In [147]:
import operator
keep = []
for diag_dict in diag_counts.values():
    sorted_diag_dict = sorted(diag_dict.items(), key=operator.itemgetter(1), reverse=True)
    keep += [name[0] for name in list(sorted_diag_dict)[:5]]

In [148]:
keepp = []
for char in char_series.index[:-1]:
    if char in keep:
        keepp.append(char)
drop = []
for char in char_series.index[:-1]:
    if char not in keepp:
        drop.append(char)

In [149]:
char_series_top_5 = char_series.drop(drop)

In [150]:
char_series_top_5.head(10)

,0,tot_feel,tot_fill,tot_hw,stopwords,polarity,cursewords,FK,uniqwords
KAT____10-Things-I-Hate-About-You_script,"leave it i said , leave it ! why did n't we ju...",18.0,0.0,87.0,97.0,-0.9984,0.0,0.0,603.0
PATRICK____10-Things-I-Hate-About-You_script,i missed you . it was a bratwurst . i was eati...,6.0,13.0,60.0,86.0,-0.9952,0.0,0.0,465.0
BIANCA____10-Things-I-Hate-About-You_script,did you change your hair ? you might wan na th...,13.0,9.0,75.0,85.0,0.9968,0.0,0.0,418.0
CAMERON____10-Things-I-Hate-About-You_script,"i do n't think so , ma'am so they tell me ... ...",8.0,8.0,58.0,71.0,0.9951,0.0,0.0,316.0
MICHAEL____10-Things-I-Hate-About-You_script,you the new guy ? c'mon . i 'm supposed to giv...,7.0,3.0,46.0,83.0,0.9984,0.0,0.0,419.0
COLE____12-Monkeys_script,"ssssst ! , what 's going on ? i did n't volunt...",24.0,89.0,120.0,92.0,0.9980,0.0,0.0,554.0
RAILLY____12-Monkeys_script,he 's been tested for drugs ? you have him in ...,24.0,68.0,117.0,94.0,0.9034,0.0,0.0,576.0
JEFFREY____12-Monkeys_script,how much you gon na pay me ? huh ? i 'd be doi...,9.0,20.0,48.0,86.0,-0.9976,0.0,0.0,547.0
RASPY VOICE____12-Monkeys_script,"hey ! who 's that ? hey , bob ... what 's your...",2.0,7.0,21.0,46.0,0.9486,0.0,0.0,149.0
FALE____12-Monkeys_script,"uh , can we help you ? excuse me . you looking...",0.0,8.0,10.0,52.0,-0.9636,0.0,0.0,177.0


In [151]:
diag_counts['10-Things-I-Hate-About-You_script']

{'KAT____10-Things-I-Hate-About-You_script': 9936,
 'PATRICK____10-Things-I-Hate-About-You_script': 7205,
 'BIANCA____10-Things-I-Hate-About-You_script': 6832,
 'CAMERON____10-Things-I-Hate-About-You_script': 4372,
 'MICHAEL____10-Things-I-Hate-About-You_script': 5198,
 'JOEY____10-Things-I-Hate-About-You_script': 2742,
 'WALTER____10-Things-I-Hate-About-You_script': 2652,
 'MANDELLA____10-Things-I-Hate-About-You_script': 1950,
 'MISS PERKY____10-Things-I-Hate-About-You_script': 1518,
 'CHASTITY____10-Things-I-Hate-About-You_script': 510,
 'SHARON____10-Things-I-Hate-About-You_script': 341,
 'BRUCE____10-Things-I-Hate-About-You_script': 356,
 'TEACHER____10-Things-I-Hate-About-You_script': 409}

In [153]:
char_series_top_5.shape[0]

3306

In [156]:
char_series.shape[0]

11075

In [157]:
#indiaroma
#indiarama 
char_series_top_5.to_csv('top5_chars.csv')

In [158]:
for index,row in char_series_top_5.iterrows():
    sentz = sentence_tokenizer.tokenize(row[0])
    char_series_top_5['tot_sents'] = len(sentz)
    

KAT____10-Things-I-Hate-About-You_script
PATRICK____10-Things-I-Hate-About-You_script
BIANCA____10-Things-I-Hate-About-You_script
CAMERON____10-Things-I-Hate-About-You_script
MICHAEL____10-Things-I-Hate-About-You_script
COLE____12-Monkeys_script
RAILLY____12-Monkeys_script
JEFFREY____12-Monkeys_script
RASPY VOICE____12-Monkeys_script
FALE____12-Monkeys_script
SOLOMON____12-Years-a-Slave_script
EPPS____12-Years-a-Slave_script
BASS____12-Years-a-Slave_script
FORD____12-Years-a-Slave_script
TIBEATS____12-Years-a-Slave_script
ARON____127-Hours_script
MEGAN____127-Hours_script
KRISTI____127-Hours_script
RANA____127-Hours_script
ERIK____127-Hours_script
JORDY____15-Minutes_script
EDDIE____15-Minutes_script
EMIL____15-Minutes_script
NICOLETTE____15-Minutes_script
HAWKINS____15-Minutes_script
MIKE____17-Again_script
ED____17-Again_script
SCARLET____17-Again_script
ALEX____17-Again_script
JULIE____17-Again_script
BOWMAN____2001-A-Space-Odyssey_script
FLOYD____2001-A-Space-Odyssey_script
POOLE__

ELLEN____Arbitrage_script
BROOKE____Arbitrage_script
MENDEZ____Argo_script
O'DONNELL____Argo_script
SIEGEL____Argo_script
CHAMBERS____Argo_script
J STAFFORD____Argo_script
ASH____Army-of-Darkness_script
ARTHUR____Army-of-Darkness_script
WISEMAN JOHN____Army-of-Darkness_script
SHEILA____Army-of-Darkness_script
EVIL ASH____Army-of-Darkness_script
ARTHUR____Arthur_script
NAOMI____Arthur_script
HOBSON____Arthur_script
VIVIENNE____Arthur_script
SUSAN____Arthur_script
RATH____Assassins_script
ELECTRA____Assassins_script
BAIN____Assassins_script
CONTRACTOR/SCREEN____Assassins_script
KETCHAM____Assassins_script
BARBARA____August-Osage-County_script
VIOLET____August-Osage-County_script
IVY____August-Osage-County_script
MATTIE FAE____August-Osage-County_script
KAREN____August-Osage-County_script
WILLS____Autumn-in-New-York_script
CHARLOTTE____Autumn-in-New-York_script
JOHN____Autumn-in-New-York_script
DOLLY____Autumn-in-New-York_script
LISA____Autumn-in-New-York_script
JAKE____Avatar_script
GRAC

EGGS____Boxtrolls,-The_script
WINNIE____Boxtrolls,-The_script
SNATCHER____Boxtrolls,-The_script
MADAME FROU FROU____Boxtrolls,-The_script
SHOE____Boxtrolls,-The_script
MASON____Boyhood_script
DAD____Boyhood_script
MOM____Boyhood_script
SAMANTHA____Boyhood_script
BILL____Boyhood_script
WALLACE____Braveheart_script
PRINCESS____Braveheart_script
WILLIAM____Braveheart_script
ROBERT____Braveheart_script
LONGSHANKS____Braveheart_script
JEFF____Breakdown_script
RED____Breakdown_script
AMY____Breakdown_script
EARL____Breakdown_script
SHERIFF BOYD____Breakdown_script
BRENDAN____Brick_script
THE BRAIN____Brick_script
PIN____Brick_script
LAURA____Brick_script
TUGGER____Brick_script
ANNIE____Bridesmaids_script
LILLIAN____Bridesmaids_script
HELEN____Bridesmaids_script
RHODES____Bridesmaids_script
MEGAN____Bridesmaids_script
HALE____Broken-Arrow_script
DEAKINS____Broken-Arrow_script
TERRY____Broken-Arrow_script
WILKINS____Broken-Arrow_script
GILES____Broken-Arrow_script
BLOOM____Brothers-Bloom,-The_

EVE____Dallas-Buyers-Club_script
RAYON____Dallas-Buyers-Club_script
RICHARD BARKLEY____Dallas-Buyers-Club_script
RICK FERRIS____Dallas-Buyers-Club_script
CLOUGH____Damned-United,-The_script
TAYLOR____Damned-United,-The_script
REVIE____Damned-United,-The_script
CUSSINS____Damned-United,-The_script
MITCHELL____Damned-United,-The_script
DUNBAR____Dances-with-Wolves_script
DANCES WITH WOLVES____Dances-with-Wolves_script
KICKING BIRD____Dances-with-Wolves_script
WIND IN HIS HAIR____Dances-with-Wolves_script
TEN BEARS____Dances-with-Wolves_script
WHITE____Dark-City_script
BUMSTEAD____Dark-City_script
SCHREBER____Dark-City_script
STROMBOLI____Dark-City_script
INSIDE____Dark-City_script
WAYNE____Dark-Knight-Rises,-The_script
BLAKE____Dark-Knight-Rises,-The_script
BANE____Dark-Knight-Rises,-The_script
GORDON____Dark-Knight-Rises,-The_script
ALFRED____Dark-Knight-Rises,-The_script
PHIL____Date-Night_script
CLARA____Date-Night_script
HOLBROOKE____Date-Night_script
LIEUTENANT MANNING____Date-Night

JOE____Fair-Game_script
JOE WILSON____Fair-Game_script
LIBBY____Fair-Game_script
PRESIDENT____Fair-Game_script
JACK____Family-Man,-The_script
KATE____Family-Man,-The_script
CASH____Family-Man,-The_script
ARNIE____Family-Man,-The_script
ALAN____Family-Man,-The_script
REED____Fantastic-Four_script
SUE____Fantastic-Four_script
BEN____Fantastic-Four_script
VICTOR____Fantastic-Four_script
JOHNNY____Fantastic-Four_script
MARGE____Fargo_script
JERRY____Fargo_script
CARL____Fargo_script
VOICE____Fargo_script
WADE____Fargo_script
STACY____Fast-Times-at-Ridgemont-High_script
DAMONE____Fast-Times-at-Ridgemont-High_script
BRAD____Fast-Times-at-Ridgemont-High_script
LINDA____Fast-Times-at-Ridgemont-High_script
THE RAT____Fast-Times-at-Ridgemont-High_script
HAZEL____Fault-in-Our-Stars,-The_script
GUS____Fault-in-Our-Stars,-The_script
FRANNIE____Fault-in-Our-Stars,-The_script
ISAAC____Fault-in-Our-Stars,-The_script
VAN HOUTEN____Fault-in-Our-Stars,-The_script
DUKE____Fear-and-Loathing-in-Las-Vegas_sc

...

COLONEL____Men-in-Black-3_script
KAY____Men-in-Black_script
JAY____Men-in-Black_script
EDWARDS____Men-in-Black_script
LAUREL____Men-in-Black_script
ZED____Men-in-Black_script
BOB____Men-Who-Stare-at-Goats,-The_script
LYN CASSADY____Men-Who-Stare-at-Goats,-The_script
BILL DJANGO____Men-Who-Stare-at-Goats,-The_script
LARRY HOOPER____Men-Who-Stare-at-Goats,-The_script
GUS LACEY____Men-Who-Stare-at-Goats,-The_script
CROCKETT____Miami-Vice_script
TUBBS____Miami-Vice_script
ISABELLA____Miami-Vice_script
JOSÉ YERO____Miami-Vice_script
GINA____Miami-Vice_script
JOE____Midnight-Cowboy_script
RATSO____Midnight-Cowboy_script
SHIRLEY____Midnight-Cowboy_script
TOWNY____Midnight-Cowboy_script
O'DANIEL____Midnight-Cowboy_script
GIL____Midnight-in-Paris_script
INEZ____Midnight-in-Paris_script
ADRIANA____Midnight-in-Paris_script
HEMINGWAY____Midnight-in-Paris_script
STEIN____Midnight-in-Paris_script
JILL____Mighty-Joe-Young_script
GREGG____Mighty-Joe-Young_script
HARRY____Mighty-Joe-Young_script
STRASS

JUDGE GARNETT____Philadelphia_script
STU____Phone-Booth_script
VOICE____Phone-Booth_script
RAMEY____Phone-Booth_script
LEON____Phone-Booth_script
FELICIA____Phone-Booth_script
SZPILMAN____Pianist,-The_script
HENRYK____Pianist,-The_script
DOROTA____Pianist,-The_script
FATHER____Pianist,-The_script
JEHUDA____Pianist,-The_script
STEWART____Piano,-The_script
BAINES____Piano,-The_script
FLORA____Piano,-The_script
AUNT MORAG____Piano,-The_script
HIRA____Piano,-The_script
DALE____Pineapple-Express_script
SAUL____Pineapple-Express_script
RED____Pineapple-Express_script
ANGIE____Pineapple-Express_script
TED____Pineapple-Express_script
JACK SPARROW____Pirates-of-the-Caribbean-Dead-Man's-Chest_script
WILL TURNER____Pirates-of-the-Caribbean-Dead-Man's-Chest_script
ELIZABETH SWANN____Pirates-of-the-Caribbean-Dead-Man's-Chest_script
GIBBS____Pirates-of-the-Caribbean-Dead-Man's-Chest_script
LORD CUTLER BECKETT____Pirates-of-the-Caribbean-Dead-Man's-Chest_script
FRY____Pitch-Black_script
JOHNS____Pitc

RALPH____Saving-Mr.-Banks_script
DON____Saving-Mr.-Banks_script
MILLER____Saving-Private-Ryan_script
REIBEN____Saving-Private-Ryan_script
SARGE____Saving-Private-Ryan_script
UPHAM____Saving-Private-Ryan_script
JACKSON____Saving-Private-Ryan_script
LAWRENCE____Saw_script
ADAM____Saw_script
TAPP____Saw_script
SING____Saw_script
JIGSAW____Saw_script
TONY____Scarface_script
LOPEZ____Scarface_script
TOAD____Scarface_script
ELVIRA____Scarface_script
OMAR____Scarface_script
SCHINDLER____Schindler's-List_script
GOETH____Schindler's-List_script
STERN____Schindler's-List_script
PFEFFERBERG____Schindler's-List_script
SCHERNER____Schindler's-List_script
SIDNEY____Scream-2_script
GALE____Scream-2_script
DEWEY____Scream-2_script
DEREK____Scream-2_script
HALLIE____Scream-2_script
GALE____Scream-3_script
SIDNEY____Scream-3_script
DEWEY____Scream-3_script
DETECTIVE KINCAID____Scream-3_script
Gale____Scream-3_script
SIDNEY____Scream_script
BILLY____Scream_script
TATUM____Scream_script
GALE____Scream_scr

SPOCK____Star-Trek-The-Motion-Picture_script
"ILIA"____Star-Trek-The-Motion-Picture_script
UHURA____Star-Trek-The-Motion-Picture_script
KIRK____Star-Trek_script
SPOCK____Star-Trek_script
SPOCK PRIME____Star-Trek_script
PIKE____Star-Trek_script
MCCOY____Star-Trek_script
JOHNNY____Starship-Troopers_script
CARMEN____Starship-Troopers_script
RASCZAK____Starship-Troopers_script
ZANDER____Starship-Troopers_script
OFFICIAL VOICE____Starship-Troopers_script
HOOKER____Sting,-The_script
GONDORFF____Sting,-The_script
LONNEGAN____Sting,-The_script
TWIST____Sting,-The_script
CALLER____Sting,-The_script
TOM____Stir-of-Echoes_script
MAGGIE____Stir-of-Echoes_script
PHILIP____Stir-of-Echoes_script
FRANK____Stir-of-Echoes_script
NEIL____Stir-of-Echoes_script
DRE____Straight-Outta-Compton_script
EAZY____Straight-Outta-Compton_script
CUBE____Straight-Outta-Compton_script
JERRY____Straight-Outta-Compton_script
SUGE____Straight-Outta-Compton_script
GUY____Strangers-on-a-Train_script
BRUNO____Strangers-on-a-

MUNTZ____Up_script
DUG____Up_script
YOUNG ELLIE____Up_script
EVEY____V-for-Vendetta_script
LEADER____V-for-Vendetta_script
FINCH____V-for-Vendetta_script
HELEN____V-for-Vendetta_script
DELIA____V-for-Vendetta_script
STAUFFENBERG____Valkyrie_script
OLBRICHT____Valkyrie_script
FROMM____Valkyrie_script
BECK____Valkyrie_script
TRESCKOW____Valkyrie_script
DAVID____Vanilla-Sky_script
SOFIA____Vanilla-Sky_script
BRIAN____Vanilla-Sky_script
MCCABE____Vanilla-Sky_script
VENTURA____Vanilla-Sky_script
GALVIN____Verdict,-The_script
MICKEY____Verdict,-The_script
ANGLE____Verdict,-The_script
CONCANNON____Verdict,-The_script
JUDGE____Verdict,-The_script
FISHER____Very-Bad-Things_script
BOYD____Very-Bad-Things_script
LIZ____Very-Bad-Things_script
ADAM____Very-Bad-Things_script
MICHAEL____Very-Bad-Things_script
PARKER____Virtuosity_script
MADISON____Virtuosity_script
COX____Virtuosity_script
LINDENMEYER____Virtuosity_script
DEANE____Virtuosity_script
WALTER____Visitor,-The_script
TAREK____Visitor,-The_

In [160]:
char_series_top_5['tot_sents'] = np.zeros(char_series_top_5.shape[0])